Planning and Optimizing a Test Campaign
=======================================
J S Jenkins
-----------

# Overview

# Terminology

|Term|Concept and Interpretation|
|-------|--------------|
| quantity | A measurable property of an object. Example: cylinder head temperature. |
| constraint | A relational expression involving at least one quantity. Example: cylinder head temperature less than 100 &deg;C. |
| scenario | A named conjunction of constraints. state of the world. Examples: idling (speed 0 m/s), level driving at highway speed (speed 28 m/s, roadway pitch 0&deg;), moderate weather conditions (ambient temperature 20 &deg;C). (For the purposes of this example, we do not require the specific constraints.) |
| situation | A conjunction of scenarios used to specify conditions of applicability of a requirement. Examples: idling under moderate weather conditions, level driving at highway speed under moderate weather conditions.
|requirement| A constraint on a quantity that applies during a disjunction of situations. Examples: While idling under moderate weather conditions or level driving at highway speed under moderate weather condititions, the cylinder head temperature shall not exceed 100 &deg;C.
|configuration| A conjunction of scenarios that stand as proxies for real-world scenarios for the purposes of testing and verification. See Situations to Configurations below.

# Workflow Example

## Sample Requirement Set

Following the terminology above, we generate a set of named quantities, a set of named scenarios, and a set of named requirements. Each requirement applies during a disjunction of unnamed situations. These situations are conjunctions of scenarios and can be represented by the sets of scenarios to be conjoined. Because a scenario is itself a conjunction of constraints, a situation expresses a set of constraints in _disjunctive normal form_.

Because constraints necessarily limit the state space, an empty situation denotes _always and everywhere_. A requirement such as "The vehicle dry mass shall not exceed 1500 kg." is interpreted to apply during an empty situation.

Each generated requirement is randomly a disjunction of one or two situations, each of which is randomly assigned a conjunction of zero to five scenarios.

Finally, each requirement is randomly assigned a quantity to constrain. For the purposes of this experiment, the particular constraint is irrelevant.

In [1]:
import json

quantities_file = "build/quantities.json"
scenarios_file = "build/scenarios.json"
requirements_file = "build/requirements.json"

quantities = json.load(fp = open(quantities_file))
scenarios = json.load(fp = open(scenarios_file))["scenarios"]
requirements = json.load(fp = open(requirements_file))["requirements"]

print(f"{len(quantities)} quantities")
print(f"{len(scenarios)} scenarios")
print(f"{len(requirements)} requirements")


50 quantities
20 scenarios
200 requirements


## Grouping by Situation

If two or more requirements apply in the same situation, it may be more efficient (and/or less error-prone) to verify them in a single activity with that situation in effect. Accordingly, the first step in the workflow is to group requirements by situation. Having done so, we obtain a map from each unique situation to the requirements that explictly apply during that situation and the quantities constrained by those requirements.

While it is impractical to conduct "blind" testing in a rigorous sense (the constraints imposed by requirements are known to testers), there is an argument that the requirements verification process should be separated into a data collection step that merely collects observations of constrained quantities regardless of the actual constraints, and a subsequent step (perhaps performed by others) that verifies conformance of those observations with the relevant constraints. In the following, we focus on the data collection activities. The verification step is a straightforward computation.

In principle, we could proceed directly to testing at this point. In practice, however, other considerations apply. These will be addressed in the following sections.

In [2]:
tests_file = "build/tests.json"
tests = json.load(open(tests_file))
print(f"{len(tests)} distinct situations")

190 distinct situations


## Situations to Configurations

In [3]:
print(f"{len(tests)} distinct configurations")

190 distinct configurations


## Costs

### Configuration Setup/Teardown Costs

### Observation Costs

## Configuration Sufficiency

By definition, if a requirement applies in a particular configuration _C_<sub>1</sub>, then it applies in any configuration _C_<sub>2</sub> for which _C_<sub>1</sub> ⊆ _C_<sub>2</sub>.
For example, if _C_<sub>1</sub> corresponds to the scenario set {_S_<sub>1</sub>, _S_<sub>2</sub>} and _C_<sub>2</sub> corresponds to the scenario set {_S_<sub>1</sub>, _S_<sub>2</sub>, _S_<sub>3</sub>}, then
the state of the world represented by by _C_<sub>2</sub> (_S_<sub>1</sub> ∧ _S_<sub>2</sub> ∧ _S_<sub>3</sub>) implies the state of the world implied by _C_<sub>1</sub> (_S_<sub>1</sub> ∧ _S_<sub>2</sub>), and therefore
any requirement that applies during _C_<sub>1</sub> also applies during _C_<sub>2</sub>.

In [4]:
tests_pruned_file = "build/tests-pruned.json"
tests_pruned = json.load(open(tests_pruned_file))
print(f"{len(tests_pruned)} distinct configurations")

148 distinct configurations


## Campaign Partitioning